# AI-Enhanced Multimodal Analysis for Rare Neurological Disorders
## Data Preprocessing, Model Development, and Visualization
This notebook processes synthetic wearable data, trains a Gradient Boosting Classifier, and visualizes results for early detection of rare neurological disorders.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# 1. Data Preprocessing
# Generate synthetic wearable data (accelerometer and heart rate)
n_samples = 1000
time = np.arange(n_samples)
accel_x = np.random.normal(0, 1, n_samples) + np.sin(time / 50)  # Simulate gait
heart_rate = np.random.normal(80, 10, n_samples)  # Simulate HR
labels = np.random.choice([0, 1], size=n_samples, p=[0.7, 0.3])  # 0: Healthy, 1: Disorder

data = pd.DataFrame({
    'time': time,
    'accel_x': accel_x,
    'heart_rate': heart_rate,
    'label': labels
})

# Handle missing values and smooth data
data['accel_x'] = data['accel_x'].fillna(data['accel_x'].mean())
data['heart_rate'] = data['heart_rate'].fillna(data['heart_rate'].mean())
data['accel_x_smoothed'] = data['accel_x'].rolling(window=10).mean()

# Feature engineering: Calculate stride variability
data['stride_variability'] = data['accel_x'].diff().abs().rolling(window=20).std()
data = data.dropna()

# 2. Model Development
X = data[['accel_x_smoothed', 'heart_rate', 'stride_variability']]
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy:.2f}')

# 3. Visualization
# Time-series plot of accelerometer data
plt.figure(figsize=(10, 6))
sns.lineplot(x='time', y='accel_x_smoothed', hue='label', data=data)
plt.title('Smoothed Accelerometer Data by Label')
plt.show()

# Interactive Plotly plot for heart rate
fig = px.line(data, x='time', y='heart_rate', color='label', title='Heart Rate Over Time')
fig.show()